In [92]:

import time
import casadi as ca
import numpy as np
from itertools import product
import matplotlib.pyplot as plt
import pdb
from scipy.stats import norm



In [372]:
class Simulator():
    
    def __init__(self,
                EV_init     = np.array([0., 13.9]),
                DT          = 0.1,
                D_NOM       = 7.,
                MODE        = 0,
                T_FINAL     = 100,
                S_FINAL     = 50, 
                NOISE_STD   = [0.6,0.6]
                ):
        
        self.ev_init=EV_init
        self.tv_init= [EV_init[0]-0.7*EV_init[1]-3., 14.0]

        self.N_modes=3
        
        self.t=0
        self.dt= DT
        self.T=T_FINAL
        self.s_f=S_FINAL
        self.d_nom=D_NOM
        self.mode=MODE
        
        self.A=np.array([[1., self.dt],[0., 1.]])
        self.B=np.array([0.,self.dt])
        
        self.a_brake=0.7*10
        self.s_stop=self.s_f-self.ev_init[1]**2/2/self.a_brake
        self.t_dec=int((self.s_stop-6.-self.tv_init[0])/self.tv_init[1]/self.dt)
        
        
        self.p_mode=np.zeros((self.N_modes,self.T))
        
        self.ev_traj=np.zeros((2,self.T+1))
        self.ev_u=np.zeros((1,self.T))
        self.ev_traj[:,0]=self.ev_init
        
        self.tv_traj=np.zeros((2,self.T+1))
        
       

        self.tv_traj[:,0]=self.tv_init
        self.p_mode[:,0]=self.N_modes**(-1)*np.ones(self.N_modes)
            
        self.u_prev=0.
        self.noise_std=NOISE_STD
            
        
    def done(self):
#         if self.mode==1:
        return self.t==self.T or (self.s_f<self.ev_traj[0,self.t]+0.1)
#             return self.t==self.T or self.ev_traj[1,self.t]<=0.1
#         else:
#             return self.t==self.T or self.tv_traj[0][0,self.t]>=self.s_f-self.d_nom-0.5
        
    def get_update_dict_i(self):      

        self._get_tv_params()
            

        update_dict={'s0': self.ev_traj[0,self.t], 'v0':self.ev_traj[1,self.t], 'u_prev':self.u_prev,
                         's_tv0': np.array(self.tv_traj[0,self.t]), 'v_tv0': np.array(self.tv_traj[1,self.t]),
                         'p_mode': self.p_mode[:,self.t]}
        
        i=int((self.s_stop-4-self.ev_traj[0,self.t])/self.dt/self.ev_traj[1,self.t])
        return i,update_dict
        
    
    def run_step(self, u_ev):
#         pdb.set_trace()
        rng=np.random.default_rng(self.t+1)
#         if self.ev_traj[0,self.t]>=self.s_f-0.35 and self.mode==1:
#             u_ev=-0.1
        self.ev_traj[:,self.t+1]=self.A@self.ev_traj[:,self.t]+self.B*u_ev
        self.ev_traj[1,self.t+1]=np.max([1e-6, self.ev_traj[1,self.t+1] ])
        self.ev_u[:,self.t]=u_ev
        self.u_prev=u_ev

        if self.mode==0 or self.t<-4+self.t_dec:
            u_tv=0.
        else:
            u_tv=-self.a_brake*1.5

        self.tv_traj[:,self.t+1]=self.A@self.tv_traj[:,self.t]+self.B*u_tv\
                                            +np.array([rng.normal(0,0.1*self.noise_std[0]), rng.normal(0,0.1*self.noise_std[1])])
        self.tv_traj[1,self.t+1]=np.max([1e-6, self.tv_traj[1,self.t+1] ])
        self.t+=1
    
    def _get_tv_params(self):

        w_tail=np.diag(np.array(self.noise_std[0:2])**(-1))@(self.tv_traj[:,self.t]-self.A@self.tv_traj[:,self.t-1])
        w_stop=np.diag(np.array(self.noise_std[0:2])**(-1))@(self.tv_traj[:,self.t]-self.A@self.tv_traj[:,self.t-1]+self.B*self.a_brake)
        
        P=np.zeros(self.N_modes)
        for j in range(self.N_modes):
            if j==0:
                P[j]=norm.pdf(w_tail[0])*norm.pdf(w_tail[1])#*max([self.ev_traj[0,self.t]-self.s_stop,0.])/(self.s_f-self.s_stop)
            elif j==1:
#                 if self.t<self.t_dec:
#                     P[j]=norm.pdf(w_tail[0])*norm.pdf(w_tail[1])#*max([self.ev_traj[0,self.t]-self.s_stop,0.])/(self.s_f-self.s_stop)
#                 else:
                P[j]=norm.pdf(w_stop[0])*norm.pdf(w_stop[1])#*max([self.ev_traj[0,self.t]-self.s_stop,0.])/(self.s_f-self.s_stop)
            else:
#                 if self.t<self.t_dec:
#                     P[j]=norm.pdf(w_tail[0])*norm.pdf(w_tail[1])#*min([(self.s_f-self.ev_traj[0,self.t])/(self.s_f-self.s_stop),1.])
#                 else:
                P[j]=norm.pdf(w_stop[0])*norm.pdf(w_stop[1])#*min([(self.s_f-self.ev_traj[0,self.t])/(self.s_f-self.s_stop),1.])

        for j in range(self.N_modes):
            if self.ev_traj[0,self.t]>=self.s_stop:
                self.p_mode[j,self.t]=self.p_mode[j,self.t-1]*P[j]/(P@self.p_mode[:,self.t-1])
                if self.p_mode[j,self.t]<=10**(-6):
                    self.p_mode[j,self.t]=10**(-6)
                if self.p_mode[j,self.t]>=1-10**(-6):
                    self.p_mode[j,self.t]=1-10**(-6)
            elif self.t>0:
                self.p_mode[j,self.t]=self.p_mode[j,self.t-1]
                

        
   

In [391]:
class SMPC_MMPreds():

    def __init__(self,
                N            =  12,
                DT           = 0.1,
                V_MIN        = 0.0,
                V_MAX        = 14.0, 
                A_MIN        = -7.0,
                A_MAX        =  3.5, 
                N_modes_MAX  =  3,
                N_TV         =  1,
                D_NOM        =  6,
                S_FINAL      =  60, 
                risk         =0.01,
                NOISE_STD    =  [0.001, 0.01], # process noise standard deviations in order [w_x, w_y, w_theta, w_v, w_TV] 
                Q = [10., .1], # weights on x, y, and v.
                R = 20.,       # weights on inputs 
                inv_cdf=[np.array([[0.02, 1.35],[0.508,0.91]]),np.array([[1.35, 2.],[0.91,0.978]])],
                fixed_risk=True,
                open_loop=False
                ):
        self.N=N
        self.DT=DT
        self.V_MIN=V_MIN
        self.V_MAX=V_MAX
        self.A_MAX=A_MAX
        self.A_MIN=A_MIN
        self.N_modes=N_modes_MAX
        self.d_nom=D_NOM
        self.s_f=S_FINAL
        self.N_TV=1
        self.risk_level=risk
        self.noise_std=NOISE_STD
        self.Q = ca.diag(Q)
        self.R = ca.diag(R)
        
        self.fixed_risk=fixed_risk
        self.ol=open_loop
        self.inv_cdfl=[]
        for i in range(len(inv_cdf)):
            
            m=(inv_cdf[i][1,1]-inv_cdf[i][1,0])/(inv_cdf[i][0,1]-inv_cdf[i][0,0])
            c=inv_cdf[i][1,0]-m*inv_cdf[i][0,0]
            self.inv_cdfl.append([m,c])
                
        self.nom_z_tv=[]
        
        self.opti=[]

        self.u_backup=0.
        self.z_curr=[]
        self.u_prev=[]
        self.slack=[]


        self.z_tv_curr=[]

        self.p_mode=[]
        
        self.policy=[]
        
        # p_opts = {'expand': True}
        # s_opts = {'max_cpu_time': 0.15, 'print_level': 0} 

        p_opts_grb = {'OutputFlag': 0, 'FeasibilityTol' : 1e-3, 'PSDTol' : 1e-3}
        s_opts_grb = {'error_on_fail':0}

        p_opts = {'expand':False, 'verbose' :False}
        s_opts = {'max_cpu_time': 10.0,  'print_level': 0}
        
        self.c_mrv=ca.DM([norm.ppf(1-self.risk_level)]*self.N_modes)
        self.c_mrp=ca.DM([(1-self.risk_level)]*self.N_modes)
        
        self.mmrisk_var=[]
        self.mmrisk_prob=[]
     
        
        for i in range(self.N):
#         for i in range(3,4):    
            # self.opti.append(ca.Opti('conic'))
            # self.opti[i].solver("gurobi", s_opts_grb, p_opts_grb)
            self.opti.append(ca.Opti())
            self.opti[i].solver("ipopt", p_opts, s_opts)
                        
            
            self.z_curr.append(self.opti[i].parameter(2))
            self.p_mode.append(self.opti[i].parameter(self.N_modes))
            self.u_prev.append(self.opti[i].parameter(1))
            self.slack.append(self.opti[i].variable(self.N_modes))
             
            self.z_tv_curr.append(self.opti[i].parameter(2))
            
            if not fixed_risk:
                self.mmrisk_var.append(self.opti[i].variable(self.N_modes))
                self.mmrisk_prob.append(self.opti[i].variable(self.N_modes))
            else:
                self.mmrisk_var.append(self.opti[i].parameter(self.N_modes))
                self.mmrisk_prob.append(self.opti[i].parameter(self.N_modes))
                self.opti[i].set_value(self.mmrisk_var[i],self.c_mrv)
                self.opti[i].set_value(self.mmrisk_prob[i],self.c_mrp)
              
                        
            self.policy.append(self._return_policy_class(i))
            self._add_constraints_and_cost(i)
            


    
    def _return_policy_class(self, i):
        
        if i == 0 or i==self.N-1 or self.ol:
            K=[self.opti[i].variable(1,2) for t in range(self.N-1)] if not self.ol else [ca.DM(1,2) for t in range(self.N-1)] 
            K_stack=[ca.diagcat(ca.DM(1,2),*[K[t] for t in range(self.N-1)])]*self.N_modes

            h=[self.opti[i].variable(1) for t in range(self.N)]
            h_stack=[ca.vertcat(*[h[t] for t in range(self.N)])]*self.N_modes
                  
        else:
   
            K=[[self.opti[i].variable(1,2) for n in range(1+(-1+self.N_modes)*int(t+1>=i))] for t in range(self.N-1)] if not self.ol else [[ca.DM(1,2) for n in range(1+(-1+self.N_modes)*int(t+1>=i))] for t in range(self.N-1)]
            K_stack=[ca.diagcat(ca.DM(1,2),*[K[t][m*int(t+1>=i)] for t in range(self.N-1)]) for m in range(self.N_modes)]
                       
            h=[[self.opti[i].variable(1) for n in range(1+(-1+self.N_modes)*int(t>=i))] for t in range(self.N)]    
            h_stack=[ca.vertcat(*[h[t][m*int(t>=i)] for t in range(self.N)]) for m in range(self.N_modes)]
            
                
        return h_stack,K_stack

    def _get_LTV_EV_dynamics(self, i):
        
        A=ca.DM([[1., self.DT],[0., 1.]])
        B=ca.DM([0., self.DT])
        E=ca.diag([0.,0.])
                
        A_pred=ca.MX(2*(self.N+1), 2)
        B_pred=ca.MX(2*(self.N+1),self.N)
        E_pred=ca.MX(2*(self.N+1),self.N*2)
        
        A_pred[:2,:]=ca.MX.eye(2)
        
        for t in range(1,self.N+1):
                A_pred[t*2:(t+1)*2,:]=A@A_pred[(t-1)*2:t*2,:]
                
                B_pred[t*2:(t+1)*2,:]=A@B_pred[(t-1)*2:t*2,:]
                B_pred[t*2:(t+1)*2,t-1]=B
                
                E_pred[t*2:(t+1)*2,:]=A@E_pred[(t-1)*2:t*2,:]
                E_pred[t*2:(t+1)*2,(t-1)*2:t*2]=E
                
        
        return A_pred,B_pred,E_pred

    def _get_tv_ATV_dyn(self,i):
                       
        T=ca.DM([[1.,self.DT],[0., 1.]])
        c=ca.DM(np.zeros((2,1)))
        E=ca.DM([[self.noise_std[0], 0.],[0., self.noise_std[1]]])
        
                
        T_tv=[ca.MX(2*(self.N+1), 2) for j in range(self.N_modes)]
        c_tv=[ca.MX(2*(self.N+1),1) for j in range(self.N_modes)]
        E_tv=[ca.MX(2*(self.N+1),self.N*2) for j in range(self.N_modes)]
        
            
        for j in range(self.N_modes):
            for t in range(self.N+1):
                if t==0:
                    T_tv[j][:2,:]=ca.DM(np.identity(2))
                else:
                    T_tv[j][t*2:(t+1)*2,:]=T@T_tv[j][(t-1)*2:t*2,:]
                    c_tv[j][t*2:(t+1)*2,:]=T@c_tv[j][(t-1)*2:t*2,:]
                    if j==0 or t<=i+3:
                        c_tv[j][t*2:(t+1)*2,:]+=c
                    else:
                        c_tv[j][t*2:(t+1)*2,:]+=ca.DM([0., self.DT])*self.A_MIN
                    
                    E_tv[j][t*2:(t+1)*2,:]=T@E_tv[j][(t-1)*2:t*2,:]
                    E_tv[j][t*2:(t+1)*2,(t-1)*2:t*2]=E
                    

        return T_tv, c_tv, E_tv 

 
        
    def _add_constraints_and_cost(self, i):
        

        [A,B,E]=self._get_LTV_EV_dynamics(i)
        [T_tv, c_tv, E_tv]=self._get_tv_ATV_dyn(i)
        
        mrv=self.mmrisk_var[i]
        mrp=self.mmrisk_prob[i]

        [h,K]=self.policy[i]
        E_perm=ca.MX(self.N*4,self.N*4)
        sl=self.slack[i]
        self.opti[i].subject_to(sl>=0)
                
        nom_z_tv_i=[T_tv[j]@self.z_tv_curr[i]+c_tv[j] for j in range(self.N_modes)]
        self.nom_z_tv.append(nom_z_tv_i)
        total_risk=0
        cost =0 
        for j in range(self.N_modes):
            cost+=1e6*sl[j]*(self.p_mode[i][j])
            self.opti[i].subject_to(self.opti[i].bounded(self.A_MIN, h[j], self.A_MAX))

            if not self.fixed_risk:
                self.opti[i].subject_to(self.opti[i].bounded(1e-6, mrv[j], 3.5))
                total_risk+=mrp[j]*self.p_mode[i][j]
                for l in range(len(self.inv_cdfl)):
                    self.opti[i].subject_to(mrp[j]<=self.inv_cdfl[l][0]*mrv[j]+self.inv_cdfl[l][1])
                    self.opti[i].subject_to(self.opti[i].bounded(0.5,mrp[j],1.))
            for t in range(1,self.N+1):

                self.opti[i].subject_to(self.opti[i].bounded(self.V_MIN, A[t*2+1,:]@self.z_curr[i]+B[t*2+1,:]@h[j], self.V_MAX))
                
                z=B[t*2,:]@K[j]@E_tv[j][:-2,:]-E_tv[j][t*2,:]*mrv[j]

                y=-1.*self.d_nom+A[t*2,:]@self.z_curr[i]+B[t*2,:]@h[j]-T_tv[j][t*2,:]@self.z_tv_curr[i]-c_tv[j][t*2,:]
                
              
                self.opti[i].subject_to(z@z.T<=y**2)
                self.opti[i].subject_to(y>=0)

            if j==self.N_modes-1 or i!=self.N-1:
                s_stop=self.s_f-.1
            else:
                s_stop=self.s_f+50
            
            v_T=A[-1,:]@self.z_curr[i]+B[-1,:]@h[j] # terminal velocity
            s_T=A[2*self.N,:]@self.z_curr[i]+B[2*self.N,:]@h[j]
            y_T=-2*self.A_MIN*(s_stop-s_T)
            
            # soc_t=ca.soc(v_T,y_T)
            # self.opti[i].subject_to(soc_t>0)

            self.opti[i].subject_to(v_T.T@v_T<=y_T+sl[j])
          
            nom_z=A@self.z_curr[i]+B@h[j]
            nom_v_diff=ca.diff(nom_z.reshape((2,-1)),1,1)[1,:].T
            nom_u_diff=ca.diff(ca.vertcat(self.u_prev[i],h[j]))

            cost+=self.p_mode[i][j]*(-self.Q[0,0]*(ca.sum1(nom_z.reshape((2,-1))[0,1:].T))\
                                     +self.Q[1,1]*nom_v_diff.T@nom_v_diff\
                                      +self.R*nom_u_diff.T@nom_u_diff+1000*(s_T-s_stop)**2)

            
        if not self.fixed_risk:
            self.opti[i].subject_to(total_risk>=1-self.risk_level)
        self.opti[i].minimize( cost )   

 
    def solve(self, i):
        st = time.time()

        try:
#             pdb.set_trace()
            sol = self.opti[i].solve()
            # Optimal solution.
            
            u_control  = np.clip(sol.value(self.policy[i][0][0][0]),self.A_MIN, self.A_MAX)
            h_opt      = [sol.value(self.policy[i][0][j]) for j in range(len(self.policy[i][0]))]
            K_opt      = [np.zeros((2*self.N, 2*self.N)) for j in range(len(self.policy[i][0]))] 
            nom_z_tv   = [sol.value(self.nom_z_tv[i][j]) for j in range(len(self.policy[i][0]))]
            
            is_opt     = True
        except:
            # Suboptimal solution (e.g. timed out).
            #self.u_backup
            sub=self.opti[i].debug
            if sub.stats()['return_status']=="Infeasible_Problem_Detected":
                is_opt=False
                u_control  = self.A_MIN
            else:
                is_opt=True
                u_control  = np.clip(sub.value(self.policy[i][0][0][0]),self.A_MIN, self.A_MAX)
                h_opt      = [sub.value(self.policy[i][0][j]) for j in range(len(self.policy[i][0]))]
                K_opt      = [np.zeros((2*self.N, 2*self.N)) for j in range(len(self.policy[i][0]))] 
                nom_z_tv   = [sub.value(self.nom_z_tv[i][j]) for j in range(len(self.policy[i][0]))]
            
#             pdb.set_trace()
#             is_opt = False

        solve_time = time.time() - st
        
        sol_dict = {}
        sol_dict['u_control']  = u_control  # control input to apply based on solution
        sol_dict['optimal']    = is_opt      # whether the solution is optimal or not
        if is_opt:
                sol_dict['h_opt']=h_opt
                sol_dict['K_opt']=K_opt
                sol_dict['nom_z_tv']=nom_z_tv
                
        sol_dict['solve_time'] = solve_time  # how long the solver took in seconds


        return sol_dict

    def update(self, i, update_dict):
        self._update_ev_initial_condition(i, *[update_dict[key] for key in ['s0', 'v0', 'u_prev']] )
        self._update_tv_initial_condition(i, *[update_dict[key] for key in ['s_tv0', 'v_tv0', 'p_mode']] )

    


    def _update_ev_initial_condition(self, i, s0, v0, u_prev):
        self.opti[i].set_value(self.z_curr[i], ca.DM([s0, v0]))
        self.opti[i].set_value(self.u_prev[i], u_prev)
        self.u_backup=u_prev
                  
    def _update_tv_initial_condition(self, i, s_tv0, v_tv0, p_mode):
        
        self.opti[i].set_value(self.z_tv_curr[i], ca.DM([s_tv0, v_tv0]))
        for j in range(self.N_modes):
            self.opti[i].set_value(self.p_mode[i][j],p_mode[j])
    


    

In [392]:

Sim=Simulator(MODE=0)
smpc = SMPC_MMPreds(DT=Sim.dt, D_NOM=Sim.d_nom, NOISE_STD=Sim.noise_std, S_FINAL=Sim.s_f, fixed_risk=False)
is_opt=False
ctr=0
feas=0
opt_list=[]

while not Sim.done():
    i,update_dict=Sim.get_update_dict_i()
    
    if i>=0:
        i=max([min([smpc.N-1,i]),1])
    else:
        i=1
    smpc.update(i,update_dict)
    sol_dict=smpc.solve(i)
    is_opt=sol_dict['optimal']
    Sim.run_step(sol_dict['u_control'])
    print("Feasible?:",is_opt, "time:", Sim.t, "i:",i, "EV acceleration:", Sim.ev_u[:,Sim.t-1])
    print("EV states:", Sim.ev_traj[:,Sim.t], "TV states:", Sim.tv_traj[:,Sim.t])
    print("Mode Probabilities:", Sim.p_mode[:,Sim.t-1])
    print("-----------------------------------------------------")
    feas+=int(is_opt)
    opt_list.append(is_opt)
    ctr+=1
    


      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |  13.37ms ( 49.88us)  13.42ms ( 50.09us)       268
       nlp_g  |  87.48ms (326.41us)  87.47ms (326.38us)       268
  nlp_grad_f  |   5.61ms (105.81us)   5.62ms (106.13us)        53
  nlp_hess_l  | 243.27ms (  4.77ms) 243.81ms (  4.78ms)        51
   nlp_jac_g  | 186.77ms (  3.52ms) 187.29ms (  3.53ms)        53
       total  | 698.83ms (698.83ms) 698.82ms (698.82ms)         1
Feasible?: True time: 1 i: 11 EV acceleration: [1.00000138]
EV states: [ 1.39       14.00000014] TV states: [-11.30926495  14.04929709]
Mode Probabilities: [0.33333333 0.33333333 0.33333333]
-----------------------------------------------------
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |  19.46ms ( 44.44us)  19.54ms ( 44.60us)       438
       nlp_g  | 128.97ms (294.45us) 128.64ms (293.70us)       438
    nlp_grad  |   1.01ms (  1.01ms)   1.01ms (  1.01ms)         1
  nlp_grad_f  |   9.31ms ( 96

In [393]:

Sim1=Simulator(MODE=2)
smpc1 = SMPC_MMPreds(DT=Sim.dt, D_NOM=Sim.d_nom, NOISE_STD=Sim.noise_std, S_FINAL=Sim.s_f, fixed_risk=False)
is_opt1=False
ctr1=0
feas1=0
opt_list1=[]
Sim1.t_dec
while not Sim1.done():
    i,update_dict=Sim1.get_update_dict_i()
    
    if i>=0:
        i=max([min([smpc1.N-1,i]),1])
    else:
        i=1
    smpc1.update(i,update_dict)
    sol_dict=smpc1.solve(i)
    is_opt1=sol_dict['optimal']
    Sim1.run_step(sol_dict['u_control'])
    print("Feasible?:",is_opt, "time:", Sim1.t, "i:",i, "EV acceleration:", Sim1.ev_u[:,Sim1.t-1])
    print("EV states:", Sim1.ev_traj[:,Sim1.t], "TV states:", Sim1.tv_traj[:,Sim1.t])
    print("Mode Probabilities:", Sim1.p_mode[:,Sim1.t-1])
    print("-----------------------------------------------------")
    feas1+=int(is_opt)
    opt_list1.append(is_opt1)
    ctr1+=1
    


      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |   7.52ms ( 28.06us)   7.53ms ( 28.09us)       268
       nlp_g  |  52.52ms (195.95us)  52.52ms (195.97us)       268
  nlp_grad_f  |   3.74ms ( 70.62us)   3.75ms ( 70.75us)        53
  nlp_hess_l  | 157.30ms (  3.08ms) 157.70ms (  3.09ms)        51
   nlp_jac_g  | 120.26ms (  2.27ms) 120.64ms (  2.28ms)        53
       total  | 443.69ms (443.69ms) 443.69ms (443.69ms)         1
Feasible?: True time: 1 i: 11 EV acceleration: [1.00000138]
EV states: [ 1.39       14.00000014] TV states: [-11.30926495  14.04929709]
Mode Probabilities: [0.33333333 0.33333333 0.33333333]
-----------------------------------------------------
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |  13.46ms ( 30.73us)  13.43ms ( 30.66us)       438
       nlp_g  |  93.28ms (212.96us)  92.95ms (212.22us)       438
    nlp_grad  | 715.00us (715.00us) 717.82us (717.82us)         1
  nlp_grad_f  |   7.22ms ( 74

In [394]:

Sim_ol=Simulator(MODE=0)
smpc_ol = SMPC_MMPreds(DT=Sim.dt, D_NOM=Sim.d_nom, NOISE_STD=Sim.noise_std, S_FINAL=Sim.s_f,open_loop=True)
is_opt_ol=False
ctr_ol=0
feas_ol=0
opt_list_ol=[]

while not Sim_ol.done():
    i,update_dict=Sim_ol.get_update_dict_i()
    
    if i>=0:
        i=max([min([smpc_ol.N-1,i]),1])
    else:
        i=1
    smpc_ol.update(i,update_dict)
    sol_dict=smpc_ol.solve(i)
    is_opt_ol=sol_dict['optimal']
    Sim_ol.run_step(sol_dict['u_control'])
    print("Feasible?:",is_opt_ol, "time:", Sim_ol.t, "i:",i, "EV acceleration:", Sim_ol.ev_u[:,Sim_ol.t-1])
    print("EV states:", Sim_ol.ev_traj[:,Sim_ol.t], "TV states:", Sim_ol.tv_traj[:,Sim_ol.t])
    print("Mode Probabilities:", Sim_ol.p_mode[:,Sim_ol.t-1])
    print("-----------------------------------------------------")
    feas_ol+=int(is_opt_ol)
    opt_list_ol.append(is_opt_ol)
    ctr_ol+=1
    


      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |   3.50ms ( 71.33us)   2.22ms ( 45.27us)        49
       nlp_g  |  12.15ms (247.92us)   7.33ms (149.58us)        49
  nlp_grad_f  |  12.22ms (643.16us)   2.89ms (152.05us)        19
  nlp_hess_l  |  29.25ms (  1.22ms)  19.60ms (816.67us)        24
   nlp_jac_g  |  32.91ms (  1.18ms)  16.48ms (588.42us)        28
       total  | 180.60ms (180.60ms)  91.49ms ( 91.49ms)         1
Feasible?: False time: 1 i: 11 EV acceleration: [-7.]
EV states: [ 1.39 13.2 ] TV states: [-11.30926495  14.04929709]
Mode Probabilities: [0.33333333 0.33333333 0.33333333]
-----------------------------------------------------
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |  21.48ms (278.94us)   4.43ms ( 57.57us)        77
       nlp_g  |  90.58ms (  1.18ms)  17.19ms (223.27us)        77
    nlp_grad  |   3.77ms (  3.77ms) 584.75us (584.75us)         1
  nlp_grad_f  |  26.90ms (768.54us)   4.76ms (1

In [396]:

Sim_ol_2=Simulator(MODE=2)
smpc_ol_2 = SMPC_MMPreds(DT=Sim.dt, D_NOM=Sim.d_nom, NOISE_STD=Sim.noise_std, S_FINAL=Sim.s_f,open_loop=True)
is_opt_ol_2=False
ctr_ol_2=0
feas_ol_2=0
opt_list_ol_2=[]

while not Sim_ol_2.done():
    i,update_dict=Sim_ol_2.get_update_dict_i()
    
    if i>=0:
        i=max([min([smpc_ol_2.N-1,i]),1])
    else:
        i=1
    smpc_ol_2.update(i,update_dict)
    sol_dict=smpc_ol_2.solve(i)
    is_opt_ol_2=sol_dict['optimal']
    Sim_ol_2.run_step(sol_dict['u_control'])
    print("Feasible?:",is_opt_ol_2, "time:", Sim_ol_2.t, "i:",i, "EV acceleration:", Sim_ol_2.ev_u[:,Sim_ol_2.t-1])
    print("EV states:", Sim_ol_2.ev_traj[:,Sim_ol_2.t], "TV states:", Sim_ol_2.tv_traj[:,Sim_ol_2.t])
    print("Mode Probabilities:", Sim_ol_2.p_mode[:,Sim_ol_2.t-1])
    print("-----------------------------------------------------")
    feas_ol_2+=int(is_opt_ol_2)
    opt_list_ol_2.append(is_opt_ol_2)
    ctr_ol_2+=1
    


      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |   3.14ms ( 64.12us)   2.01ms ( 40.94us)        49
       nlp_g  |  27.14ms (553.88us)   9.47ms (193.21us)        49
  nlp_grad_f  |   2.41ms (127.00us)   1.79ms ( 94.27us)        19
  nlp_hess_l  |  32.03ms (  1.33ms)  19.11ms (796.14us)        24
   nlp_jac_g  |  47.46ms (  1.70ms)  19.00ms (678.61us)        28
       total  | 201.02ms (201.02ms)  95.43ms ( 95.43ms)         1
Feasible?: False time: 1 i: 11 EV acceleration: [-7.]
EV states: [ 1.39 13.2 ] TV states: [-11.30926495  14.04929709]
Mode Probabilities: [0.33333333 0.33333333 0.33333333]
-----------------------------------------------------
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |  17.70ms (229.82us)   3.89ms ( 50.54us)        77
       nlp_g  |  84.71ms (  1.10ms)  16.78ms (217.87us)        77
    nlp_grad  |   4.35ms (  4.35ms) 698.26us (698.26us)         1
  nlp_grad_f  |  14.34ms (409.57us)   3.34ms ( 

In [406]:
## Plotting
%matplotlib notebook

import matplotlib.pyplot as plt

Sim0=Sim

fig=plt.figure(figsize=(12, 10))
# fig=plt.figure()
a11=plt.subplot(421)#(421)
plt.plot(Sim0.ev_traj[0,:Sim0.t], color='g',linestyle='-', label='ego_mm')
plt.plot(Sim_ol.ev_traj[0,:Sim_ol.t], color='tab:brown',linestyle='-', label='ego_ol')
plt.plot(Sim0.s_stop*np.ones(Sim0.t), color='b',linewidth=2.0, linestyle='--', label= 's_dec')
plt.plot(Sim0.tv_traj[0,:Sim0.t], color='tab:orange',linestyle='-', label= 'target')
plt.plot(Sim0.s_f*np.ones(Sim0.t), color='r',linewidth=2.0, linestyle='-', label= 's_f')

plt.xlim(0,Sim0.t)

plt.ylabel('s',fontsize=18)
a11.set_title("mode=0 (TL:Yellow, TV:No brake)", fontsize=20)
plt.legend()

a12=plt.subplot(422)#(421)
plt.plot(Sim1.ev_traj[0,:Sim1.t], color='g',linestyle='-', label='ego_mm')
plt.plot(Sim_ol_2.ev_traj[0,:Sim_ol_2.t], color='tab:brown',linestyle='-', label='ego_ol')
plt.plot(Sim1.s_stop*np.ones(Sim1.t), color='b',linewidth=2.0, linestyle='--', label= 's_dec')
plt.plot(Sim1.tv_traj[0,:Sim1.t], color='tab:orange',linestyle='-', label= 'target')
plt.plot(Sim1.s_f*np.ones(Sim1.t), color='r',linewidth=2.0, linestyle='-', label= 's_f')

plt.xlim(0,Sim1.t)

a12.set_title("mode=2 (TL:Red, TV:brake)", fontsize=20)

a21=plt.subplot(423)#(423)
plt.plot(Sim0.ev_traj[1,:Sim0.t], color='g',linestyle='-', label='ego_mm')
plt.plot(Sim_ol.ev_traj[1,:Sim_ol.t], color='tab:brown',linestyle='-', label='ego_ol')
plt.plot(Sim0.tv_traj[1,:Sim0.t], color='tab:orange',linestyle='-', label= 'target')
plt.plot(14.0*np.ones(Sim0.t), color='r',linewidth=2.0,linestyle='-', label= 'v_max')
plt.plot(0.0*np.ones(Sim0.t), color='y',linewidth=2.0,linestyle='-', label= 'v_min')

plt.ylabel('v',fontsize=18)
plt.xlim(0,Sim0.t)
a22=plt.subplot(424)#(423)
plt.plot(Sim1.ev_traj[1,:Sim1.t], color='g',linestyle='-', label='ego_mm')
plt.plot(Sim_ol_2.ev_traj[1,:Sim_ol_2.t], color='tab:brown',linestyle='-', label='ego_ol')
plt.plot(Sim1.tv_traj[1,:Sim1.t], color='tab:orange',linestyle='-', label= 'target')
plt.plot(14.0*np.ones(Sim1.t), color='r',linewidth=2.0,linestyle='-', label= 'v_max')
plt.plot(0.0*np.ones(Sim1.t), color='y',linewidth=2.0,linestyle='-', label= 'v_min')

plt.xlim(0,Sim1.t)

a31=plt.subplot(425)

t_array0=np.arange(Sim0.t)
plt.plot(smpc.A_MAX*np.ones(Sim0.t), color='r',linewidth=2.0,linestyle='-', label= 'a_max')
plt.plot(Sim0.ev_u[0,:Sim0.t], color='g',linestyle='-', label='ego_mm')
plt.plot(Sim_ol.ev_u[0,:Sim_ol.t], color='tab:brown',linestyle='-', label='ego_ol')
plt.plot(smpc.A_MIN*np.ones(Sim0.t), color='y',linewidth=2.0,linestyle='-', label= 'a_min')
plt.ylabel('a',fontsize=18)
plt.xlim(0,Sim0.t)



a32=plt.subplot(426)

t_array0=np.arange(Sim1.t)
plt.plot(smpc.A_MAX*np.ones(Sim1.t), color='r',linewidth=2.0,linestyle='-', label= 'a_max')
plt.plot(Sim1.ev_u[0,:Sim1.t], color='g',linestyle='-', label='ego_mm')
plt.plot(Sim_ol_2.ev_u[0,:Sim_ol_2.t], color='tab:brown',linestyle='-', label='ego_ol')
plt.plot(smpc.A_MIN*np.ones(Sim1.t), color='y',linewidth=2.0,linestyle='-', label= 'a_min')

plt.xlim(0,Sim1.t)
# plt.xlabel('time')

a41=plt.subplot(427)
plt.plot(Sim0.p_mode[0,:Sim0.t], color='g',linestyle='-', label='mode=0')
plt.plot(Sim0.p_mode[1,:Sim0.t], color='m',linestyle='-', label='mode=1')
plt.plot(Sim0.p_mode[2,:Sim0.t], color='b',linestyle='-', label='mode=2')
plt.xlabel('time', fontsize=18)
plt.ylabel('Probabilities',fontsize=18)
plt.xlim(0,Sim0.t)
plt.legend()
a42=plt.subplot(428)
plt.plot(Sim1.p_mode[0,:Sim1.t], color='g',linestyle='-', label='mode=0')
plt.plot(Sim1.p_mode[1,:Sim1.t], color='m',linestyle='-', label='mode=1')
plt.plot(Sim1.p_mode[2,:Sim1.t], color='b',linestyle='-', label='mode=2')
plt.xlabel('time', fontsize=18)
plt.legend()
plt.xlim(0,Sim1.t)


plt.show()

plt.savefig('tl_scenario.png')


<IPython.core.display.Javascript object>